# Download Dataset Directly From Kaggle

In [1]:
import opendatasets as od

In [2]:
md = "https://www.kaggle.com/datasets/omkargurav/face-mask-dataset"
od.download(md,force = True)

100%|███████████████████████████████████████████████████████████████████████████████| 163M/163M [02:17<00:00, 1.25MB/s]


# Import The Dataset

In [1]:
import os

In [21]:
with_mask_imgs = os.listdir(r"D:\CodeClause\Mask Detection Project\face-mask-dataset\data\with_mask")
without_mask_imgs = os.listdir(r"D:\CodeClause\Mask Detection Project\face-mask-dataset\data\without_mask")
data_categories = os.listdir(r"D:\CodeClause\Mask Detection Project\face-mask-dataset\data")

Labels = [i for i in range(len(data_categories))]
Labels_dict= dict(zip(data_categories,Labels))

print(data_categories)
print(Labels_dict)
print(Labels)

['without_mask', 'with_mask']
{'without_mask': 0, 'with_mask': 1}
[0, 1]


In [22]:
len(with_mask_imgs)

3725

In [23]:
len(without_mask_imgs)

3828

# Doing Preprocessing

In [6]:
import cv2 
import numpy as np

data = []
all_labels = []

for category in data_categories:
    folder_path = os.path.join (r"D:\CodeClause\Mask Detection Project\face-mask-dataset\data", category)
    img_names = os.listdir(folder_path)
   
    for img_name in img_names:
        img_path = os.path.join(folder_path, img_name)
        img = cv2.imread (img_path)
 
        img_RGP = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img_resized = cv2.resize(img_RGP, (128 , 128))
        img_array = np.array(img_resized)
        data.append(img_array)
        all_labels.append(Labels_dict[category])

In [7]:
x = np.array(data)
x = np.reshape(x,(x.shape[0],128,128,3))
y = np.array(all_labels)

In [8]:
type(x)

numpy.ndarray

In [9]:
print(x.shape)
print(y.shape)

(7553, 128, 128, 3)
(7553,)


In [10]:
print(y)

[0 0 0 ... 1 1 1]


In [11]:
all_labels[3825:3830]

[0, 0, 0, 1, 1]

## Building the Model

In [13]:
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras

In [14]:
num_classes = 2

model = keras.Sequential()

model.add(keras.layers.Conv2D(128,(3,3), activation='relu', input_shape=(128,128,3)))
model.add(keras.layers.MaxPooling2D(pool_size=(2,2)))


model.add(keras.layers.Conv2D(64,(3,3), activation='relu'))
model.add(keras.layers.MaxPooling2D(pool_size=(2,2)))

model.add(keras.layers.Flatten())

model.add(keras.layers.Dense(128, activation='relu'))
model.add(keras.layers.Dropout(0.5))

model.add(keras.layers.Dense(64, activation='relu'))
model.add(keras.layers.Dropout(0.5))


model.add(keras.layers.Dense(num_classes, activation='sigmoid'))

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [15]:
X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size=0.2, random_state=2)

In [16]:
X_train_scaled = X_train / 255

X_test_scaled = X_test / 255

In [17]:
history = model.fit(X_train_scaled, Y_train, validation_split=0.2, epochs=15)

Epoch 1/15
152/152 [==============================] - 19s 64ms/step - loss: 0.5848 - accuracy: 0.7076 - val_loss: 0.4038 - val_accuracy: 0.8420
Epoch 2/15
152/152 [==============================] - 9s 58ms/step - loss: 0.3583 - accuracy: 0.8585 - val_loss: 0.3015 - val_accuracy: 0.8834
Epoch 3/15
152/152 [==============================] - 9s 58ms/step - loss: 0.3290 - accuracy: 0.8701 - val_loss: 0.2821 - val_accuracy: 0.8825
Epoch 4/15
152/152 [==============================] - 9s 58ms/step - loss: 0.3103 - accuracy: 0.8699 - val_loss: 0.2842 - val_accuracy: 0.8875
Epoch 5/15
152/152 [==============================] - 9s 58ms/step - loss: 0.2695 - accuracy: 0.8955 - val_loss: 0.2955 - val_accuracy: 0.8916
Epoch 6/15
152/152 [==============================] - 9s 58ms/step - loss: 0.2715 - accuracy: 0.8918 - val_loss: 0.2829 - val_accuracy: 0.8743
Epoch 7/15
152/152 [==============================] - 9s 59ms/step - loss: 0.2683 - accuracy: 0.8928 - val_loss: 0.2618 - val_accuracy: 0.893

In [18]:
print(model.evaluate(X_test_scaled, Y_test))

48/48 [==============================] - 1s 24ms/step - loss: 0.1924 - accuracy: 0.9385
[0.192354217171669, 0.93845134973526]


In [24]:
model.save("Mask Detecion Model")

INFO:tensorflow:Assets written to: Mask Detecion Model\assets


# Real Time Mask Detection

In [26]:
video = cv2.VideoCapture(0)
counter = 0
haar = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

labels_dict = {1:"Mask", 0:"No Mask"}
color = {1 : (0,255,0) , 0 : (0,0,255)}


while(True):

        ret , img = video.read()
        imgRGP = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        faces = haar.detectMultiScale(imgRGP, scaleFactor=1.3, minNeighbors=5)
        for x,y,w,h in faces:

            
            face_img = imgRGP[ y:y+w , x:x+w]
            resized = cv2.resize(face_img,(128,128))
            normalized = resized / 255
            reshaped = np.reshape(normalized,(1 , 128 ,128 , 3))
            result = model.predict(reshaped)
            label = np.argmax(result , axis = 1)[0]
            
            cv2.rectangle(img,(x , y), (x+w,y+h), color[label],2)
            cv2.rectangle(img,(x , y-40),(x+w,y), color[label],-1)
            cv2.putText(img, labels_dict[label], (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255),2)

        cv2.imshow('LIVE', img)
       

        if cv2.waitKey(20) & 0xFF == ord('d'):
            break


video.release()   
cv2.destroyAllWindows()

# Mask Detecion From Image

In [27]:
img = cv2.imread('image5.JPG')

rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

haar = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
faces = haar.detectMultiScale(rgb, scaleFactor=1.2, minNeighbors=2,flags=cv2.CASCADE_FIND_BIGGEST_OBJECT)

for (x, y, w, h) in faces:
    face = rgb[y:y+h, x:x+w]
    face_resized = cv2.resize(face, (128, 128)) 
    face_normalized = face_resized / 255
    face_expanded = np.reshape(face_normalized,(1 , 128 ,128 , 3)) # Add a batch dimension

    prediction = model.predict(face_expanded)

    if prediction[0][0] > prediction[0][1]:
        cv2.rectangle(img, (x, y), (x+w, y+h), (0, 0, 255), thickness=2)
        cv2.putText(img, 'No Mask', (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), thickness=2)
    else:
        cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), thickness=2)
        cv2.putText(img, 'Mask', (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), thickness=2)

cv2.imshow('Detected Faces', img)
cv2.waitKey(0)

-1